[View in Colaboratory](https://colab.research.google.com/github/rushivarun/lyrics_generator_using-LSTM/blob/master/rap_king1_0_train.ipynb)

In [0]:
import numpy as np
import pandas as pd

In [4]:
from google.colab import files 
uploaded = files.upload()

Saving drake-songs.csv to drake-songs.csv


In [0]:
data = pd.read_csv('drake-songs.csv')

In [6]:
print(data.head())

   Unnamed: 0  Unnamed: 0.1             song        album  \
0           0             0       successful  so-far-gone   
1           1             1  best-i-ever-had  so-far-gone   
2           2             2           uptown  so-far-gone   
3           3             3      im-going-in  so-far-gone   
4           4             4         the-calm  so-far-gone   

                                              lyrics  
0  Money, money, cars, cars|-|Clothes, clothes, t...  
1  You know alot of girls be|-|Thinkin' my songs ...  
2  Yeah..|-|Uh huhhh|-|Uh, Hardly Home But Always...  
3  Hello mothafucka, hey hi how ya durrn'?|-|It's...  
4  Uh, I'm just so far gone, october's own, Pleas...  


In [7]:
import re
text = ''

for index, row in data['lyrics'].iteritems():
    cleaned = str(row).lower().replace(' ', '\n')
    text = text + " ".join(re.findall(r"[a-z']+", cleaned))
    
len(text)

367372

In [8]:
tokens = re.findall(r"[a-z'\s]", text)

chars = sorted(list(set(tokens)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 28


CREATE TRAINING SEQUENCES


In [9]:
maxlen = 40
step = 1
sentences = []
next_char = []
for i in range(0, len(text)-maxlen, step):
  sentences.append(text[i:i+maxlen])
  next_char.append(text[i+maxlen])
print("number of training sequences =",len(sentences))

number of training sequences = 367332


VECTORIZATION

In [0]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype = np.bool)
y = np.zeros((len(sentences), len(chars)))


ONE HOT ENCODING


In [0]:
for i, sentence in enumerate(sentences):
  for j, char in enumerate(sentence):
    x[i, j, char_indices[char]] = 1
  y[i, char_indices[next_char[i]]] = 1

BUILDING MODEL

In [12]:
from keras.models import Sequential
from keras.callbacks import LambdaCallback
from keras.layers import Dense, Activation
from keras.optimizers import Adam, RMSprop
from keras.utils.data_utils import get_file
from keras.layers import LSTM

Using TensorFlow backend.


In [13]:
print("model in action....")
model = Sequential()
model.add(LSTM(128,input_shape=(maxlen,len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer =RMSprop(lr=0.01))


model in action....


In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               80384     
_________________________________________________________________
dense_1 (Dense)              (None, 28)                3612      
_________________________________________________________________
activation_1 (Activation)    (None, 28)                0         
Total params: 83,996
Trainable params: 83,996
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(x, y, batch_size=128, epochs = 11)

Epoch 1/11
367332/367332 [==============================] - 320s 871us/step - loss: 2.9066
Epoch 2/11
367332/367332 [==============================] - 319s 870us/step - loss: 2.8427
Epoch 3/11
367332/367332 [==============================] - 320s 870us/step - loss: 2.8419
Epoch 4/11
367332/367332 [==============================] - 320s 871us/step - loss: 2.8428
Epoch 5/11
367332/367332 [==============================] - 320s 871us/step - loss: 2.8411
Epoch 6/11
367332/367332 [==============================] - 320s 872us/step - loss: 2.8410
Epoch 7/11
367332/367332 [==============================] - 319s 869us/step - loss: 3.0069
Epoch 8/11
367332/367332 [==============================] - 320s 870us/step - loss: 3.0297
Epoch 9/11
367332/367332 [==============================] - 320s 871us/step - loss: 3.0308
Epoch 10/11
367332/367332 [==============================] - 319s 869us/step - loss: 3.0303
Epoch 11/11
367332/367332 [==============================] - 319s 870us/step - loss: 3.03

In [0]:
saved = model.save_weights("model.h5")

In [0]:
downloaded = files.download("model.h5")